# Pytorch使用基础

本文主要参考[pytorch中文文档&教程](https://pytorch.apachecn.org/docs/1.2/)。完全参照官方文档，主要涉及一些最基本的内容：

- pytorch的基本概念
- 通过简单神经网络示例进一步了解概念
- 理解torch.nn包

最后一部分最为重要，因为这是在自己运用pytorch时最关键的。

## pytorch快速入门

pytorch作为NumPy的替代品，可以利用GPU的性能进行计算；可作为一个高灵活性、速度快的深度学习平台。

Tensor（张量）类似于NumPy的ndarray，但还可以在GPU上使用来加速计算。因此经常看到把numpy的数组包装为tensor再运算。tensor的操作和numpy中的数组操作类似，不再赘述，详见官网。

In [ ]:
from __future__ import print_function
import torch
# 构建一个 5x3 的矩阵, 未初始化的:
x = torch.Tensor(5, 3)
print(x)
# 构建一个随机初始化的矩阵:
x = torch.rand(5, 3)
print(x)
# 获得 size:
print(x.size())
# 加法
y = torch.rand(5, 3)
print(x + y)

print(torch.add(x, y))

result = torch.Tensor(5, 3)
torch.add(x, y, out = result)
print(result)

# 可以用类似Numpy的索引来处理所有的张量！
print(x[:, 1])

# 改变大小: 如果你想要去改变tensor的大小, 可以使用 torch.view:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

# 转换一个 Torch Tensor 为 NumPy 数组
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)
# 查看 numpy 数组是如何改变的.a和b是绑定的
a.add_(1)
print(a)
print(b)

# 看改变 np 数组之后 Torch Tensor 是如何自动改变的
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out = a)
print(a)
print(b)

# 可以使用 .cuda 方法将 Tensors 在GPU上运行.
# 只要在  CUDA 是可用的情况下, 我们可以运行这段代码
if torch.cuda.is_available():
    b = b.cuda()
    print(b + b)

由于和numpy的紧密联系，因此pytorch的张量和numpy数组可以很方便的转换。

In [2]:
a = torch.ones(5)
b = a.numpy()
print(b)

import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[1. 1. 1. 1. 1.]
[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


CPU上的所有张量(CharTensor除外)都支持与Numpy的相互转换。

张量要在GPU上计算，需要主动从CPU移动到GPU上。张量可以使用.to方法移动到任何设备（device）上

In [4]:
x = torch.randn(1)
print(x)
print(x.item())
# 当GPU可用时,我们可以运行以下代码
# 我们将使用`torch.device`来将tensor移入和移出GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # 直接在GPU上创建tensor
    x = x.to(device)                       # 或者使用`.to("cuda")`方法
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # `.to`也能在移动时改变dtype

tensor([-0.8521])
-0.8521100282669067
tensor([0.1479], device='cuda:0')
tensor([0.1479], dtype=torch.float64)


### Autograd自动求导

PyTorch中，所有神经网络的核心是 autograd 包。

autograd 包为**张量上的所有操作**提供了**自动求导机制**。它是一个在运行时定义（define-by-run）的框架，这意味着**反向传播是根据代码如何运行来决定的**，并且每次迭代可以是不同的.

**torch.Tensor** 是这个包的核心类。如果设置它的属性 **.requires_grad** 为 True，那么它将会**追踪对于该张量的所有操作**。当完成计算后可以通过**调用 .backward()，来自动计算所有的梯度**。这个张量的所有梯度将会自动累加到.grad属性.

还有一个类对于autograd的实现非常重要：**Function**。Tensor 和 Function 互相连接生成了一个无圈图(acyclic graph)，它编码了完整的计算历史。

In [ ]:
import torch
from torch.autograd import Variable

# 创建 variable（变量）:

x = Variable(torch.ones(2, 2), requires_grad = True)
print(x)

# variable（变量）的操作:

y = x + 2
print(y)

# y 由操作创建,所以它有 grad_fn 属性.

print(y.grad_fn)

# y 的更多操作

z = y * y * 3
out = z.mean()

print(z, out)

# 梯度 我们现在开始了解反向传播, out.backward() 与 out.backward(torch.Tensor([1.0])) 这样的方式一样

out.backward()

# 但因 d(out)/dx 的梯度
# 你应该得到一个 4.5 的矩阵. 让我们推导出 out Variable “o”. 我们有 o=1/4∑izi, zi=3(xi+2)^2 和 zi∣∣xi=1=27. 因此, ∂o/∂xi=32(xi+2), 所以 ∂o∂xi∣∣xi=1=9/2=4.5.
print(x.grad)

# 你可以使用自动求导来做很多有趣的事情
x = torch.randn(3)
# torch.norm(input, p=2) → float
#
# 返回输入张量input 的p 范数。
#
# 参数：
#     input (Tensor) – 输入张量
#     p (float,optional) – 范数计算中的幂指数值
x = Variable(x, requires_grad = True)

y = x * 2
print(y.data)
print(y.data.norm())

while y.data.norm() < 1000:
    y = y * 2

print(y)

gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

# 上面整个用计算图的思路去理解就很简单了

### 神经网络介绍

神经网络可以使用 **torch.nn** 包构建.
 
autograd 实现了反向传播功能, 但是直接用来写深度学习的代码在很多情况下还是稍显复杂,torch.nn 是专门为神经网络设计的模块化接口. nn 构建于 Autograd 之上, 可用来定义和运行神经网络. nn.Module 是 nn 中最重要的类, 可把它看成是一个网络的封装, 包含网络各层定义以及 forward 方法, 
调用 forward(input) 方法, 可返回前向传播的结果.

一个典型的神经网络训练过程如下:

- 定义具有一些可学习参数(或权重)的神经网络
- 迭代输入数据集
- 通过网络处理输入
- 计算损失(输出的预测值与实际值之间的距离)
- 将梯度传播回网络
- 更新网络的权重, 通常使用一个简单的更新规则: weight = weight - learning_rate * gradient

In [6]:
# 定义一个网络:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 卷积层 '1'表示输入图片为单通道, '6'表示输出通道数, '5'表示卷积核为5*5
        # 核心
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全连接层: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #在由多个输入平面组成的输入信号上应用2D最大池化.
        # (2, 2) 代表的是池化操作的步幅
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果大小是正方形, 则只能指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批量维度外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 数据处理

通常来说，当必须处理图像、文本、音频或视频数据时，可以使用python标准库将数据**加载到numpy数组**里。然后将这个数组**转化成torch.*Tensor**。

- 对于图片，有Pillow，OpenCV等包可以使用；
- 对于音频，有scipy和librosa等包可以使用；
- 对于文本，不管是原生python的或者是基于Cython的文本，可以使用NLTK和SpaCy。

特别对于视觉方面，我们创建了一个包，名字叫torchvision，其中包含了针对Imagenet、CIFAR10、MNIST等常用数据集的数据加载器（data loaders），还有对图片数据变形的操作，即torchvision.datasets和torch.utils.data.DataLoader。

但是像网络上下载的纯数据，想要顺利地输入到自己定义的模型中，可能这一步就要花些时间来处理了。

在训练中，想要利用GPU需要做一些处理：

与将一个张量传递给GPU一样，可以这样**将神经网络转移到GPU上**。

如果我们有cuda可用的话，让我们首先定义第一个设备为可见cuda设备：

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


然后这些方法将递归遍历所有模块，并将它们的参数和缓冲区转换为CUDA张量。请记住，我们不得不将**输入和目标**在**每一步都送入GPU**（inputs, labels = inputs.to(device), labels.to(device)）。

In [8]:
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 跟着例子学习PyTorch

通过例子了解pytorch的基本概念。

PyTorch的核心是提供两个主要功能：

- n维张量，类似于numpy，但可以在GPU上运行；
- 自动区分以构建和训练神经网络。

### 张量

先使用 numpy 实现网络（更具体地可以参考:[numpy部分的手写神经网络](https://github.com/OuyangWenyu/hydrus/blob/master/3-numpy-examples/neural_network.py)）。

Numpy 提供了一个n维的数组对象, 并提供了许多操纵这个数组对象的函数。Numpy 是科学计算的通用框架; Numpy 数组没有计算图, 也没有深度学习, 也没有梯度下降等方法实现的接口。但是可以很容易地使用 numpy 生成随机数据，并将产生的数据传入双层的神经网络,并实现这个网络的正向传播和反向传播:

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np

# N是批尺寸参数；D_in是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：计算预测值y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 计算并显示loss（损失）
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 反向传播，计算w1、w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

Numpy 是一个伟大的框架, 但它不能利用 GPU 加速它数值计算，而对于现代的深度神经网络, GPU 往往是提供 50倍或更大的加速,所以 numpy 不足以满足现在深度学习的需求。

PyTorch提供了Tensor，其在概念上与 numpy 数组相同，也是一个n维数组, 不过PyTorch 提供了很多能在这些 Tensor 上操作的函数。

像 numpy 数组一样, PyTorch Tensor 也和numpy的数组对象一样不了解深度学习,计算图和梯度下降，它们只是科学计算的通用工具；

然而不像 numpy, PyTorch Tensor 可以利用 GPU 加速他们的数字计算。

要在 GPU 上运行 PyTorch 张量, 只需将其转换为新的数据类型.

将 PyTorch Tensor 生成的随机数据传入双层的神经网络. 就像上面的 numpy 例子一样,
我们需要手动实现网络的正向传播和反向传播:

In [ ]:
# -*- coding: utf-8 -*-

import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N是批尺寸大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：计算预测值y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 反向传播，计算w1、w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

### 自动求导

对于小型的两层网络而言，手动实现反向传递并不重要，但对于大型的复杂网络而言，这变得非常麻烦。

幸运的是，我们可以使用**自动微分** 来自动计算神经网络中的反向传播。PyTorch中的 autograd软件包提供了这个功能。使用autograd时，您的网络正向传递将定义一个 计算图；图中的节点为张量，图中的边为从输入张量产生输出张量的函数。通过该图进行反向传播，可以轻松计算梯度。

这听起来很复杂，在实践中非常简单。**每个张量代表计算图中的一个节点**。如果 x是一个张量，并且有 x.requires_grad=True，那么x.grad就是另一个张量，代表着x相对于某个标量值的梯度。

通过使用PyTorch张量和autograd来实现网络就不再需要手动实现网络的反向传播：

In [ ]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N是批尺寸大小；D_in是输入维度；
# H是隐藏层维度；D_out是输出维度 
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出数据，将requires_grad置为False，意味着我们不需要在反向传播时候计算这些值的梯度
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 产生随机权重tensor，将requires_grad设置为True，意味着我们希望在反向传播时候计算这些值的梯度
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：使用tensor的操作计算预测值y。
    # 由于w1和w2有requires_grad=True,涉及这些张量的操作将让PyTorch构建计算图，从而允许自动计算梯度。
    # 由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 计算并输出loss
    # loss是一个形状为(1,)的张量
    # loss.item()是这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 使用autograd计算反向传播,这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()

    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 反向传播之后手动将梯度置零，原因在后面第三节nn到底是什么一节中简单补充说明
        w1.grad.zero_()
        w2.grad.zero_()

在底层，每一个原始的**自动求导运算**实际上是**两个在Tensor上运行的函数**。其中，**forward**函数计算从输入Tensors获得的输出Tensors。而**backward**函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。

在PyTorch中，可以很容易地通过定义**torch.autograd.Function**的子类并实现**forward和backward函数**，来**定义自己的自动求导运算**。然后，我们可以通过**构造实例**并**像调用函数一样调用它**，并传递包含输入数据的张量。

In [ ]:
# -*- coding: utf-8 -*-
import torch


class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，并完成张量的正向和反向传播。
    """

    @staticmethod
    def forward(ctx, input):
        """
        在前向传播中，我们收到包含输入和返回的张量包含输出的张量。 
        ctx是可以使用的上下文对象存储信息以进行向后计算。 
        您可以使用ctx.save_for_backward方法缓存任意对象，以便反向传播使用。
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N是批尺寸大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 为了使用我们的方法，我们调用Function.apply方法。 我们将其命名为“ relu”。
    relu = MyReLU.apply

    # 正向传播：使用张量上的操作来计算输出值y;
    # 我们使用自定义的自动求导操作来计算 RELU.
    y_pred = relu(x.mm(w1)).mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 使用autograd计算反向传播过程。
    loss.backward()

    # 用梯度下降更新权重
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 在反向传播之后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

### torch.nn模块

计算图和autograd是定义复杂运算符并自动采用导数的非常强大的范例。但是对于大型神经网络，原始的autograd可能会有点太低了。

在TensorFlow中，诸如Keras， TensorFlow-Slim和TFLearn之类的软件包在原始计算图上提供了更高级别的抽象接口，这些封装对构建神经网络很有用。

在PyTorch中，该nn程序包达到了相同的目的。该nn 包定义了**一组Modules**，它们**大致等效于神经网络层**。模块接收输入张量并计算输出张量，但也可以保持内部状态，例如包含可学习参数的张量。该nn软件包还定义了一组有用的**损失函数**，这些函数通常在训练神经网络时使用。

In [ ]:
# -*- coding: utf-8 -*-
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nn包还包含常用的损失函数的定义
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量
    y_pred = model(x)

    # 计算并打印损失。我们传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 反向传播之前清零梯度
    model.zero_grad()

    # 反向传播：计算模型的损失对所有可学习参数的梯度
    # 在内部，每个模块的参数存储在requires_grad=True的张量中
    # 因此这个调用将计算模型中所有可学习参数的梯度
    loss.backward()

    # 使用梯度下降更新权重
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

在实践中，我们经常使用更复杂的优化器（例如AdaGrad，RMSProp，Adam等）来训练神经网络。

PyTorch中的软件包optim抽象了优化算法的思想，并提供了常用优化算法的实现。

在此示例中，我们将像之前一样使用nn包来定义模型，但是用optim包提供的Adam算法来优化模型：

In [ ]:
# -*- coding: utf-8 -*-
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包定义模型和损失函数
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用optim包定义优化器（Optimizer）。Optimizer将会为我们更新模型的权重
# 这里我们使用Adam优化方法；optim包还包含了许多别的优化算法
# Adam构造函数的第一个参数告诉优化器应该更新哪些张量
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 前向传播：通过像模型输入x计算预测的y
    y_pred = model(x)

    # 计算并输出loss
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)。
    # 这是因为默认情况下，每当调用.backward（）时，渐变都会累积在缓冲区中（即不会被覆盖）
    # 有关更多详细信息，请查看torch.autograd.backward的文档。
    optimizer.zero_grad()

    # 反向传播：根据模型的参数计算loss的梯度
    loss.backward()

    # 调用Optimizer的step函数使它所有参数更新
    optimizer.step()

有时，您将需要**指定比一系列现有模块更复杂的模型**。在这些情况下，您可以通过**继承nn.Module和定义一个forward来定义自己的模型**，这个forward模块可以使用其他模块或在Tensors上的其他自动求导运算来接收输入Tensors并生成输出Tensors。

In [ ]:
# -*- coding: utf-8 -*-
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意的（可微分的）操作。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 通过实例化上面定义的类来构建我们的模型
model = TwoLayerNet(D_in, H, D_out)

# 构造损失函数和优化器
# SGD构造函数中对model.parameters()的调用
# 将包含模型的一部分，即两个nn.Linear模块的可学习参数
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 前向传播：通过向模型传递x计算预测值y
    y_pred = model(x)

    # 计算并输出loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 清零梯度，反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## torch.nn到底是什么

因为nn包是使用pytorch时最重要的工具，因此这里结合官网《torch.nn到底是什么》一文，认真理解一遍。这部分是本文最重要的环节。示例用的是MNIST数据。

2.用PyTorch中的**nn.Linear可以代替手动定义和初始化self.weights和self.bias以及计算xb @ self.weights + self.bias**, 因为nn.Linear可以完成这些操作。 PyTorch中预设了很多类型的神经网络层，使用它们可以极大的简化我们的代码，通常还会带来速度上的提升。

3.PyTorch还有一个包含很多优化算法的包————torch.optim。我们可以使用优化器中的step方法执行前向传播过程中的步骤来替换手动更新每个参数。
这个方法将允许我们替换之前手动编写的优化步骤。

### 获取数据

In [10]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

该数据集采用numpy数组格式，并已使用pickle存储，pickle是一个用来把数据序列化为python特定格式的库。

每一幅图像都是28 x 28的，并被拉平成长度为784(=28x28)的一行。 我们以其中一个为例展示一下，首先需要将这个一行的数据重新变形为一个2d的数据。

In [15]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [17]:
from matplotlib import pyplot
import numpy as np

%matplotlib inline
# pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

(50000, 784)


PyTorch使用torch.tensor，而不是numpy数组，所以我们需要将数据转换。

In [18]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
# n是样本个数50000,c是每个样本长度
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


### 不使用nn构建神经网络

使用PyTorch提供的**创建随机数填充或全零填充张量**的方法，**初始化**一个简单线性模型的**权重和偏置**。 这两个都是普通的张量，但它们有一个特殊的附加条件：**设置需要计算梯度的参数为True**。这样PyTorch就会记录所有与这个张量相关的运算，使其能在反向传播阶段自动计算梯度。

对于weights而言，由于我们希望初始化张量过程中存在梯度，所以我们在初始化之后**设置requires_grad**。（注意：**尾缀为_的方法**在PyTorch中表示这个操作会被立即被执行。）

注意：以[Xavier初始化方法](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)（每个元素都除以1/sqrt(n)）为例来对权重进行初始化。

In [19]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
# 尾缀为_的方法在PyTorch中表示这个操作会被立即被执行
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

多亏了PyTorch具有**自动梯度计算**功能，我们可以使用Python中**任何标准函数**（或者可调用对象）来创建模型！ 因此，这里编写一个普通的矩阵乘法和广播加法建立一个简单的线性模型，以及一个激活函数（使用一个log_softmax函数）。 请记住：尽管Pytorch提供了许多预先编写好的损失函数、激活函数等等，仍然可以使用纯python轻松**实现你自己的函数**。 Pytorch甚至可以自动地为你的函数创建快速的GPU代码或向量化的CPU代码。

In [20]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)   # @表示点积运算符

bs = 64  # 一批数据个数 batchsize

xb = x_train[0:bs]  # 从x获取一小批数据
preds = model(xb)  # 预测值
preds[0], preds.shape
# 一次前向计算的结果
print(preds[0], preds.shape)

tensor([-2.0541, -2.3306, -1.9161, -2.0688, -2.8124, -2.4089, -2.2310, -2.6536,
        -2.7899, -2.1908], grad_fn=<SelectBackward>) torch.Size([64, 10])


注意**张量preds不仅包括了张量值，还包括了梯度函数**（也就是因为这样，才能直接调用backward函数反向传播）。这个梯度函数我们可以在后面的反向传播阶段用到。

接下来实现一个负的对数似然函数（Negative log-likehood）作为损失函数（同样也使用纯python实现）：

In [21]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll
yb = y_train[0:bs]
print(loss_func(preds, yb))

tensor(2.3581, grad_fn=<NegBackward>)


再来实现一个用来计算模型准确率的函数。对于每次预测，我们规定如果预测结果中概率最大的数字和图片实际对应的数字是相同的，那么这次预测就是正确的。

先来看一下被随机初始化的模型的准确率，这样我们就可以看到损失值降低的时候准确率是否提高了。

In [22]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

print(accuracy(preds, yb))

tensor(0.0469)


运行一个完整的训练步骤了，每次迭代会进行以下几个操作：

- 从全部数据中选择一小批数据（大小为bs，即batchsize）
- 使用模型进行预测
- 计算当前预测的损失值
- 使用loss.backward()更新模型中的梯度，在这个例子中，更新的是weights和bias

**利用计算出的梯度更新权值和偏置项**，因为我们**不希望这一步的操作被用于下一次迭代的梯度计算**，所以我们在**torch.no_grad()这个上下文管理器中完成**。

将**梯度设置为0**，来为下一次循环做准备。因为梯度将会记录所有已经执行过的运算，即loss.backward()会将梯度变化值直接与变量已有值进行累加，而不是替换变量原有的值，这样设计的原因可以参考[PyTorch中在反向传播前为什么要手动将梯度清零？](https://www.zhihu.com/question/303070254)。总的来说，一般每次batch计算前都会把梯度清零，只计算本次的梯度，然后更新一次网络，但是也可以执行梯度累加，即之前batch计算的梯度累加到后来的梯度上，然后根据累加的梯度更新参数，过一些batch循环之后再将梯度清零，这样相当于变相地扩大了batchsize，使训练速度快一些，对显存不够的机器来说是一个trick。

In [26]:
from IPython.core.debugger import set_trace

lr = 0.5  # 学习率
epochs = 2  # 训练的轮数

for epoch in range(epochs):
    # n是样本数量，bs是batchsize，每个epoch内训练的次数是batch的个数 
    for i in range((n - 1) // bs + 1):
#         set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        # 反向传播之后还需要更新权重：       
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            # 设置梯度为0          
            weights.grad.zero_()
            bias.grad.zero_()

print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0599, grad_fn=<NegBackward>) tensor(1.)


总结一下之前所有步骤：获取数据——定义网络——初始化参数——定义激活函数、损失函数、准确率等——epoch和batch两层循环-对每个batch样本前向计算，损失计算，反向传播、权重更新、梯度归0——所有循环结束输出结果

### 使用torch.nn.functional

接下来使用nn的一系列模块重构代码，使代码更简洁。

第一步也是最简单的一步，是使用torch.nn.functional（通过会在引用时用F表示）中的函数**替换我们自己的激活函数和损失函数**使代码变得更短。 这个模块包含了torch.nn库中的所有函数（这个库的其它部分是各种类），所以在这个模块中还会找到其它便于建立神经网络的函数，比如池化函数。（模块中还包含卷积函数，线性函数等等，不过在后面的内容中我们会看到，这些操作使用库中的其它部分会更好。）

对负对数似然损失和对数柔性最大值(softmax)激活函数，PyTorch有一个结合了这两个函数的简单函数F.cross_entropy可供使用，这样我们就可以删掉模型中的激活函数。

注意在model函数中我们**不再调用log_softmax**（F中的loss_func应该是有固定的调用模式的，所以要在外面调用loss_func计算，参数是model输出和实际值）。可确认一下损失值和准确率与之前相同。

In [27]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0599, grad_fn=<NllLossBackward>) tensor(1.)


### 使用torch.nn.Module

接下来用nn.Moduel和nn.Parameter来完成一个更加清晰简洁的训练循环。继承nn.Module(它是一个**能够跟踪状态的类**)。

新建一个类，实现**存储权重，偏置和前向传播**步骤中所有用到方法。

nn.Module包含了许多属性和方法（比如.parameters()和.zero_grad()），它是一个PyTorch中特有的概念，是一个会经常用到的类。**不要和Python中module（m小写）混淆**，module是指一个可以被引入的Python代码文件。

nn.Module对象的使用方式很像函数（例如它们是**可调用的**），PyTorch将会**自动调用定义的forward函数**（意味着有__call__函数）。

首先实例化一个对象，然后像函数一样使用它即可。

In [28]:
from torch import nn

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb @ self.weights + self.bias
    
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

tensor(2.3953, grad_fn=<NllLossBackward>)


之前在每个训练循环中，我们通过变量名对每个变量的值进行更新，并手动的将每个变量的梯度置为0，现在我们可以利用**model.parameters()和model.zero_grad()** （这两个都是PyTorch定义在nn.Module中的）使这些步骤变得更加简洁并且更不容易忘记更新部分参数，尤其是模型很复杂的情况：

In [29]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                # 最后两步简化                
                for p in model.parameters():
                    p -= p.grad * lr
                # 一步到位设置权重和偏置梯度为0                
                model.zero_grad()

fit()
print(loss_func(model(xb), yb))

tensor(0.0822, grad_fn=<NllLossBackward>)


### 使用torch.nn.Linear

用PyTorch中的**nn.Linear代替手动定义和初始化self.weights和self.bias以及计算xb @ self.weights + self.bias**, 因为nn.Linear可以完成这些操作。 PyTorch中预设了很多类型的神经网络层，使用它们可以极大的简化我们的代码，通常还会带来速度上的提升。

In [31]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        # Linear简化初始化参数和一步前向计算过程
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)
    
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

fit()

print(loss_func(model(xb), yb))

tensor(2.3877, grad_fn=<NllLossBackward>)
tensor(0.0818, grad_fn=<NllLossBackward>)


### 使用torch.optim

PyTorch还有一个包含很多**优化算法**的包——torch.optim。我们可以使用优化器中的**step方法执行前向传播过程中的步骤**来替换手动更新每个参数。

In [33]:
from torch import optim

def get_model():
    """这里将建立模型和优化器的步骤放在一起方便复用"""
    model = Mnist_Logistic()
    # 使用SGD随机梯度下降算法，其参数是model的paramters以及学习率
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        # 一步到位更新参数，设置0梯度也是利用opt
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(2.2764, grad_fn=<NllLossBackward>)
tensor(0.0810, grad_fn=<NllLossBackward>)


到此，先总结一小波，对应训练神经网络的所有步骤里都用到了什么：

定义神经网络（继承Module来定义网络，通过**optim定义优化算法**）——初始化参数及定义前向计算函数（使用Linear完成初始化参数以及一步前向计算定义）——定义激活函数和损失函数（使用nn.functional的函数可以一步到位直接计算出损失值）——两层循环中调用model前向计算（调用Module的子类对象就像调用函数一样，会自动调用forward函数）——计算损失值（调用loss计算函数，参数是model输出和实际值）——反向传播（直接调用loss张量的backward函数即可）——更新参数（使用optim的step函数）——梯度设为0（使用optim的zero_grad方法）

接下来，在前面的数据处理环节也可以重构。

### 使用Dataset

Pytorch包含一个**Dataset抽象类**。Dataset可以是任何东西，但它始终包含一个 **__len__函数** （通过Python中的标准函数len调用）和一个**用来索引到内容中的__getitem__函数**。接下来以创建Dataset的**自定义子类**FacialLandmarkDataset为例进行介绍。

PyTorch中的**TensorDataset是一个封装了张量的Dataset**。通过定义长度和索引的方式，是我们可以对张量的第一维进行迭代，索引和切片。这将使我们在训练中，获取同一行中的自变量和因变量更加容易。可以把x_train和y_train中的数据合并成一个简单的TensorDataset，这样就可以方便的进行迭代和切片操作。

In [35]:
from torch.utils.data import TensorDataset

# 定义TensorDataset对象
train_ds = TensorDataset(x_train, y_train)
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        # 直接索引、切片 
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0816, grad_fn=<NllLossBackward>)


### 使用DataLoader

PyTorch的DataLoader负责**批量数据管理**，你可以使用任意的Dataset创建一个DataLoader。DataLoader使得**对批量数据的迭代更容易**。DataLoader**自动的为我们提供每一小批量的数据**来代替切片的方式train_ds[i*bs:i*bs+bs].

In [36]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

for xb,yb in train_dl:
    pred = model(xb)
    
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0806, grad_fn=<NllLossBackward>)


小结一下，后面增加的两个类是可以使数据的处理，包括索引、切片和分batch更容易的类。

接下来是一些提供模型效率所需的基本特征。

### 增加验证集

在实际训练中始终应该有一个验证集来确认模型是否过拟合。

**打乱训练数据的顺序通常是避免不同批数据中存在相关性和过拟合的重要步骤**。但是另一方面，**无论是否打乱顺序计算出的验证集损失值都是一样的**。鉴于打乱顺序还会消耗额外的时间，所以打乱**验证集数据**是没有任何意义的。

我们在验证集上用到的每批数据的数量是训练集的两倍，这是因为**在验证集上不需要进行反向传播，这样就会占用较小的内存**（因为它并不需要储存梯度）。我们利用了这一点，**使用了更大的batchsize，更快的计算出了损失值**（如果显存不够就要用小一点的batchsize了）。

我们将会在每轮(epoch)结束后计算并输出验证集上的损失值。

（注意：在**训练前我们总是会调用model.train()函数**，在**推断之前调用model.eval()函数**，因为这些会被nn.BatchNorm2d，nn.Dropout等层使用，确保在不同阶段的准确性。）

In [37]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

model, opt = get_model()

for epoch in range(epochs):
    # 在每个epoch内，训练前都要调用 model.train()函数
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
    # 在每个epoch内，训练后，计算loss前，都要调用 model.eval()函数
    model.eval()
    with torch.no_grad():
        # 在no_grad环境内计算loss
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

0 tensor(0.4151)
1 tensor(0.3024)


### 编写fit()和get_data()函数

我们在计算训练集和验证集上的损失值时执行了差不多的过程两次，因此我们将这部分代码提炼成一个函数loss_batch，用来计算每个批的损失值。

我们为训练集传递一个优化器参数来执行反向传播。对于验证集我们不传优化器参数，这样就不会执行反向传播。

fit执行了训练模型的必要操作，并在每轮(epoch)结束后计算模型在训练集和测试集上的损失。

get_data返回训练集和验证集需要使用到的dataloaders。

现在，我们只需要三行代码就可以获取数据、拟合模型了。

In [38]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)
        
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.35463817160129546
1 0.2867066876769066


### 使用GPU

首先检查一下的GPU是否可以被PyTorch调用：

In [39]:
print(torch.cuda.is_available())

True


接下来，新建一个设备对象：

In [40]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

然后更新一下preprocess函数将批运算移到GPU上计算

In [41]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)


train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

NameError: name 'WrappedDataLoader' is not defined

最后，我们可以把模型移动到GPU上。

In [ ]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)